# CIFAR 10 Normalization Values

In [ ]:
mean=[x/255.0 for x in [125.3, 123.0, 113.9]]
print(mean)
std=[x/255.0 for x in [63.0, 62.1, 66.7]]
print(std)

# Let's see _how to load_ ``MNIST`` data

- Import the **required** packages
- Load the data 
- Do **required** normalization

In [ ]:
import captum
from captum.attr import IntegratedGradients, Occlusion, LayerGradCam, LayerAttribution
from captum.attr import visualization as viz
from matplotlib.colors import LinearSegmentedColormap


def plot_maps_using_captum(model, FinalData, Labels, classnames, data_name, saliency_dict, method_captions, p, model_name="", cm=None, vis_sign='positive'):
    id = len(saliency_dict)
    D = np.asarray(range(95, 105, 1))  # Using: MNIST 395 - 405, fMNIST: 10 - 20
    L = Labels[D[:]]

    model.eval()
    with torch.no_grad():
        output = model(FinalData) # We don't need to unsqueeze(1) if we already have that dimension as in color image
    
    preds = output.data.max(1, keepdim=True)[1]
    print(preds.shape)
    print(torch.flatten(preds[D[:]]))
    
    fig, axes = plt.subplots(nrows = 10, ncols = id+1, sharex = "all", figsize = (12,12), squeeze=False)

    for i in range(10):
        
        sample = saliency_dict[0][D[i]]
        
        if np.squeeze(sample).ndim == 3:
            attribution = np.transpose(sample, (1,2,0))
        else:
            attribution = np.expand_dims(sample, axis=2)
        
        img = FinalData[D[i]].numpy()
        
#         if np.squeeze(img).ndim == 3:
        img = np.transpose(img, (1,2,0))
    
        # for imagenet
#         mean = np.array([0.485, 0.456, 0.406])
#         std = np.array([0.229, 0.224, 0.225])
        
        # for MNIST
#         mean =np.array([0.1307])
#         std = np.array([0.3081])

        # for CIFAR 10
    
        mean = np.array([0.5, 0.5, 0.5])
        std = np.array([0.5, 0.5, 0.5])
    
#         mean=[x/255.0 for x in [125.3, 123.0, 113.9]]
#         std=[x/255.0 for x in [63.0, 62.1, 66.7]]
        
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        plt_fig, plt_axis = viz.visualize_image_attr(attribution,
                                                     img,
                                                     "original_image",
                                                     "all",
                                                     (fig.number, axes[i, 0]),
                                                     title="",
                                                     use_pyplot = False)
        
        pred_title = str(classnames[preds[D[i]].item()])
        axes[i,0].set_ylabel("Pr: "+pred_title, fontsize=8)

    for method_id in range(id):

        saliency = saliency_dict[method_id]

        for i in range(10):

            sample = saliency_dict[method_id][D[i]]
            
            if np.squeeze(sample).ndim == 3:
                attribution = np.transpose(sample, (1,2,0))
            else:
                attribution = np.expand_dims(sample, axis=2)
            
            fig_tuple = fig.number, axes[i, method_id + 1]
            
            
            plt_fig, plt_axis = viz.visualize_image_attr(attribution,
                                                         img,
                                                         "heat_map",
                                                         vis_sign,
                                                         fig_tuple,
                                                         cmap = cm,
                                                         title="",
                                                         use_pyplot = False)

    axes[0, 0].set_title('Image/Data', fontsize='medium')


    for method_id in range(id):
        axes[0, method_id + 1].set_title(method_captions[method_id], fontsize='medium')

    plt.show()

    path = os.path.join('./Plots/Real/', "method_research_"+model_name+'_'+data_name+'_usingCapt_'+vis_sign+'_'+str(p) )
    print(path)
#     fig.savefig(path+'.svg', transparent=True, bbox_inches='tight', pad_inches=0)
    fig.savefig(path+'.pdf', format='pdf', dpi=300)
#     fig.savefig(path+'.png', format='png', dpi=300)
#     print('Plots Saved...', path)
    plt.close(fig)


# Finalize MNIST Maps

*  Choose appropriate *dataset* and path to *saliency*
*  Select which methods you want to _visualize_ saliency for 
*  Load the data and saliency.
*  Do the necessary **post-processing** for the saliency maps.

In [ ]:
from methods.method_research_utilities import load_fmnist_saliency_data, generate_post_processed_maps
# !pip install numpy==1.16.1
%matplotlib inline

method_titles = ["GD", "ONLY.IG", "ONLY.M", "GDAsc.IG","GDAsc.M", "M.GDAsc.IG", "M.GDAsc.M","Wt.P.IG", "Wt.P.M", "IG", "CaptIG", "L.IG"]

Dataset = {0: 'mnist', 1: 'fmnist', 2: 'cifar10', 3: 'imgnet'}
data = 0 # 
fname_common = "method_research_"+Dataset[data]+"_unsoftmaxed" #_vgg_customized"
# fname_common = "method_research_"+Dataset[data]+"_unsoftmaxed_vgg_customized" # for Fashion MNIST
# fname_common = "method_research_"+Dataset[data]+"_unsoftmaxed_vgg_customized_new"

# For MNIST use random_seeds = range(10)
list_of_saliency_dicts, titles = generate_post_processed_maps(data, fname_common, method_list=["GD", "ONLY.IG", "ONLY.M", "GDAsc", "M.GDAsc", "Wt.P"], \
                                                      random_seeds=list(range(10)), viz=False, scale_flag=False)

# Visualize MNIST/Fashion MNIST/CIFAR10 Maps

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pylab as plt
import numpy as np
import torch
import torchvision
import torchvision.models as models
import os
%matplotlib inline
from models import SanityCheckCNN, VGGNet, FashionCustomizedCNN

from methods.method_research_utilities import load_mnist_saliency_data, load_fmnist_saliency_data,\
    load_cifar10_saliency_data, load_cifar10_original_saliency_data, load_imagenet_saliency_data

FinalData, Labels, dataLoaderSal, categories = load_cifar10_original_saliency_data()
print(FinalData.shape)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for p in range(1, 11):
#     model = SanityCheckCNN()
#     model = FashionCustomizedCNN()
    model = VGGNet(3, 10) # CIFAR model

#     path = './models and saliencies/method_research_mnist_model_'+str(p)+'.pth'
    #     path = './models and saliencies/method_research_fmnist_model_vgg_customized_'+str(restart)+'.pth'
    path = './models and saliencies/method_research_cifar_model_vgg_customized_new_'+str(p)+'.pth'

    model_dict = torch.load(path, map_location=device)  # with good components
    model.load_state_dict(model_dict)
    model.to(device)
    print('Model loaded from:', path)

    model.eval()

    colors = {'positive': 'Reds', 'absolute_value': 'Reds', 'all': LinearSegmentedColormap.from_list("BlWhRd", ["blue", "white","red"])}
    sign = 'absolute_value'

    plot_maps_using_captum(model, FinalData, Labels, categories, Dataset[data], list_of_saliency_dicts[p-1], titles, p, cm=colors[sign], vis_sign=sign)

# Visualize Fashion MNIST

- Load the ``maps``
- Preprocess them using absolute/ReLU/min-max normalization
- Plot them **_in suitable visualization_**

In [ ]:
from methods.method_research_utilities import load_fmnist_saliency_data, generate_post_processed_maps
# !pip install numpy==1.16.1
%matplotlib inline

method_titles = ["GD", "ONLY.IG", "ONLY.M", "GDAsc.IG","GDAsc.M", "M.GDAsc.IG", "M.GDAsc.M","Wt.P.IG", "Wt.P.M", "IG", "CaptIG", "L.IG"]

Dataset = {0: 'mnist', 1: 'fmnist', 2: 'cifar10', 3: 'imgnet'}
data = 1 # 
fname_common = "method_research_"+Dataset[data]+"_unsoftmaxed_vgg_customized"
# visualize_maps(1, fname_common, 2)
list_of_saliency_dicts = generate_post_processed_maps(data, fname_common, random_seeds=1, viz=True, scale_flag=False)

# Finalize CIFAR 10 Maps

In [ ]:
from methods.method_research_utilities import load_cifar10_saliency_data, generate_post_processed_maps
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt
%matplotlib inline

method_titles = ["GD", "ONLY.IG", "ONLY.M", "GDAsc.IG","GDAsc.M", "M.GDAsc.IG", "M.GDAsc.M","Wt.P.IG", "Wt.P.M", "IG", "CaptIG", "L.IG"]

Dataset = {0: 'mnist', 1: 'fmnist', 2: 'cifar10', 3: 'imgnet'}
data = 2 # 
fname_common = "method_research_"+Dataset[data]+"_unsoftmaxed_vgg_customized_new"
# visualize_maps(1, fname_common, 2)
list_of_saliency_dicts = generate_post_processed_maps(data, fname_common, method_list=["GD", "ONLY.IG", "ONLY.M", "GDAsc", "M.GDAsc", "Wt.P"],\
                                                      random_seeds=list(range(1, 11)), viz=False, scale_flag=False)

# [Metric](www.google.com) Evaluation 

## ``MNIST``/``FMNIST``/``CIFAR 10`` Dataset
[To know about all the metrics here](www.google.com)

- ``sample = np.clip(sample, -1, 1)``
- ``sample = (sample + 1.0)/2.0``
- ``sample = sample.transpose(1,2,0)``

In [ ]:
from methods.method_research_utilities import load_mnist_saliency_data, load_fmnist_saliency_data, generate_post_processed_maps
import os
import numpy as np

%matplotlib inline

from methods.saliency_evaluation_methods import calcAllMetric, calc_metric_for_real_data

def do_metric_evaluation(data):
    
    Dataset = {0: 'mnist', 1: 'fmnist', 2: 'cifar10', 3: 'imgnet'}
    
    DatasetDict = {"mnist": load_mnist_saliency_data, "fmnist": load_fmnist_saliency_data}
    
    # Generate/Obtain Data
    
    FinalData, Labels = DatasetDict[Dataset[data]]()
    
    print('Original Data Shape:', FinalData.shape)

    print('Test Data Shape:', Labels.shape)
    fname_common = "method_research_"+Dataset[data]+"_unsoftmaxed"  #_vgg_customized

    prefix = "method_research_"+Dataset[data]

    filenamepath = "./models and saliencies"
    filenamepath = os.path.join(filenamepath, 'saliency')


    list_of_saliency_dicts = generate_post_processed_maps(data, \
                                fname_common, random_seeds=1, viz=False, scale_flag=False)


    '''
    k: interpretability method key
    total_labels: 10
    '''
    total_labels = 10
    method_titles = ["GD", "ONLY.IG", "ONLY.M", "GDAsc.IG","GDAsc.M", "M.GDAsc.IG", "M.GDAsc.M","Wt.P.IG", "Wt.P.M"]

    out_metric = {i: {k : [] for k in method_titles} for i in range(total_labels)}

    for saliency_dict_item in list_of_saliency_dicts:
        for k, v in saliency_dict_item.items():


            # Call for metric evaluation here ....

            if k < len(method_titles):
                out = calc_metric_for_real_data(saliency_dict_item[k], FinalData, Labels, total_labels, method_titles[k])

                print('Calculated Metric Shape:')
                for lab in range(total_labels):
                    out_metric[lab][method_titles[k]].append(np.array(out[lab]))

                    print(np.array(out_metric[lab][method_titles[k]]).shape)

            else:
                print(f'Key Value: {k}, not under our consideration')


    # Save the metrics 

    # np.savez(filenamepath+"/"+prefix+"out_0_all_metric_ReLUed_map.npz", **out_0_metric)  # absolute data
    # np.savez(filenamepath+"/"+prefix+"out_1_all_metric_ReLUed_map.npz", **out_1_metric)

    for labl in range(total_labels):
        np.savez(filenamepath+"/"+prefix+"_out_"+str(labl)+"_all_metrics_corrected_preAbsolute_map_all_same_mask.npz", **out_metric[labl])

    for k in method_titles:
        print(f"Method: {k}")

        for lab in range(total_labels):
            print(np.array(out_metric[lab][k]).shape)
            print(np.mean(out_metric[lab][k], axis=1))
            print(np.mean(np.mean(out_metric[lab][k], axis=1), axis=0))
    
    print('Metric Evaluation is Done...')
    


In [ ]:
do_metric_evaluation(data=0)

# Visualize Metric Evaluation Results

In [ ]:

# Generate Box Plot Using Evaluation #

import numpy as np
import pandas as pd
import os

import tkinter
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
import sys
# from statannot import add_stat_annotation

%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

from myscripts.rar_plot_helper import create_per_method_result
from sklearn.preprocessing import MaxAbsScaler

# sns.set(style="darkgrid", palette="pastel", font_scale=1.9)
sns.set(style="whitegrid", font_scale=1.5)
# sns.set_theme()


def visualize_metric_eval_results(data, save=False):
    
    Dataset = {0: 'mnist', 1: 'fmnist', 2: 'cifar', 3: 'imagenet'}

    class_labels = {'mnist': ['zero', 'one', 'two', 'three',
                           'four', 'five', 'six', 'seven', 'eight', 'nine'], \
                   'fmnist': ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', \
                          'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], \
                   'cifar10' : ['plane', 'car', 'bird', 'cat', \
                           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']}

    classes = class_labels[Dataset[data]]
    print(f'Classes: {classes}')


    prefix = "method_research_"+Dataset[data]

    filenamepath = "./models and saliencies"
    filenamepath = os.path.join(filenamepath, 'saliency')

    out = {}
    npz = {}

    for labl in range(10):
        out[labl] = filenamepath+"/"+prefix+"_out_"+str(labl)+"_all_metrics_corrected_preAbsolute_map_all_same_mask.npz"
        print('Loading metric results from: {}'.format(out[labl]))
        npz[labl] = np.load(out[labl])

    frames = []

    metric_names = ["Pearson Correlation", "Spearman Rank", "Weighted Jaccard", "Relevance Percent", "Structural Similarity", "Mean Square Error"]

    for k in npz[0].files:

        for labl in range(10):

            for metric_id in range(6):
                out_dict = {}
                per_metric_result = npz[labl][k][:, :, metric_id]

                out_dict[k] = per_metric_result.flatten()
                df = pd.DataFrame.from_dict(out_dict)
                df = df.rename(columns={k: "score"})
                df.insert(0, "class", class_labels[Dataset[data]][labl])
                df.insert(2, "method", k)
                df.insert(3, "metric", metric_names[metric_id])
                frames.append(df)
        #         print(df)

    my_df = pd.concat(frames, axis = 0)
    print(my_df)

    my_df.to_csv('./Experimental Results/Real/'+Dataset[data]+'_corrected_preAbsolute_maps_all_same_mask_all_metrics_results.csv', index=False)

    
    seven_class = ['#b2182b','#ef8a62','#fddbc7','#f7f7f7','#d1e5f0','#67a9cf','#2166ac']

    eight_class = ['#d73027','#f46d43','#fdae61','#fee090','#e0f3f8','#abd9e9','#74add1','#4575b4']

    sns.set_palette(palette=eight_class)

    f, axes = plt.subplots(4, 1, figsize=(10,10))
    
   
    count = 0
    
    print(npz[0].files)
    
    desired_methods = ["GD", "ONLY.IG", 'M.GDAsc.M'] #, "Wt.P.M"]  #'M.GDAsc.IG', "Wt.P.IG"
    desired_metrics = ["Spearman Rank", "Weighted Jaccard", "Structural Similarity", "Mean Square Error"] #"Pearson Correlation"
    user_legends = ["GRAD", "IG", "GGIG"]
    
    user_titles = {"Spearman Rank": "Spearman Rank Correlation", "Weighted Jaccard": "Weighted Jaccard Similarity", 
                   "Structural Similarity": "Structural Similarity", "Mean Square Error": "Normalized Mean Square Error"}

    limits = {"Pearson Correlation": (0.3, 0.85), "Spearman Rank": (0.2,0.8), 
              "Weighted Jaccard": (0.0,0.61), "Structural Similarity": (0.0, 0.61), "Mean Square Error": (0.0, 0.81)}
    
    for metric in desired_metrics:
        data_to_plot = my_df[my_df["metric"].isin([metric])]
        
        if metric == "Mean Square Error":
            dff = data_to_plot.copy()
            m_value = dff["score"].abs().max()
            data_to_plot.loc[:, 'score'] = 1.0 - (data_to_plot.loc[:, 'score']/m_value)
        
        data_to_plot = data_to_plot[data_to_plot["method"].isin(desired_methods)]
        

        print(data_to_plot.shape)
        
        print('Metric Name:{}'.format(metric))
        print(data_to_plot.groupby(['method']).describe().score[['50%','std']])

        g1 = sns.boxplot(x="class", y="score",linewidth=0.5, width=0.6,
                    hue="method",
                    data=data_to_plot, ax=axes[count])

        axes[count].set(ylim=limits[metric])
        axes[count].set(yticks=np.arange(limits[metric][0], limits[metric][1], 0.2))
        
        axes[count].set_title(user_titles[metric], fontsize=14)

        if count < 3:
            axes[count].set_xlabel('')
            axes[count].xaxis.set_ticklabels([])
        else:
            axes[count].set_xlabel('MNIST Digits', fontsize=12)
        
        x_ticks = axes[count].get_xticklabels()
        labels = axes[count].yaxis.get_ticklabels()
        
        yticks = np.round(np.arange(limits[metric][0], limits[metric][1], 0.2), 1)
        
        g1.set_yticklabels(yticks, size = 15)
            
        axes[count].set_xticklabels(x_ticks, rotation=0, fontsize=12)

        axes[count].set_ylabel('Score', fontsize=12)

        handles, labels = axes[count].get_legend_handles_labels()
        axes[count].legend(handles[:], user_legends, prop={'size': 10}, ncol=3, loc=1, title="", borderaxespad=0.)
       
        count += 1

    f.tight_layout()
    plt.subplots_adjust(wspace=0.01, hspace=0.2)

    plt.show()
    
    if save:
        f.savefig('./Plots/Real/'+Dataset[data]+'_nips2022_corrected_preAbsolute_all_metrics_2.svg', transparent=True, bbox_inches='tight', pad_inches=0, dpi=300)

In [ ]:
visualize_metric_eval_results(data=0, save=True)